In [ ]:
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster(threads_per_worker=1)

In [ ]:
from dask.distributed import Client, wait
import time

import dask
import dask_cudf
import dask.dataframe as dd

import pandas as pd

import cudf
import numpy as np

import pandas.testing

from dask_cuml.neighbors import NearestNeighbors as cumlNN


In [ ]:
client = Client(cluster)
client

In [ ]:
def create_df(f, m, n):
    X = np.random.rand(m, n)
    ret = cudf.DataFrame([(i,X[:,i].astype(np.float32)) for i in range(n)], 
                        index = cudf.dataframe.RangeIndex(f*m, f*m+m, 1))
    
    print(str(ret))
    return ret

def get_meta(df):
    ret =  df.iloc[:0]
    return ret

In [ ]:
workers = client.has_what().keys()
workers

In [ ]:
# Per gpu/worker
train_m = 10000 
train_n = 1000

In [ ]:
search_m = 10000
search_k = 15

In [ ]:
%%time

# Create dfs on each worker (gpu)
dfs = [client.submit(create_df, n, train_m, train_n, workers = [worker])
       for worker, n in list(zip(workers, list(range(len(workers)))))]

# Wait for completion
wait(dfs)

meta = client.submit(get_meta, dfs[0]).result()

In [ ]:
lr = cumlNN()

In [ ]:
%%time
X_df = dask_cudf.from_delayed(dfs, meta=meta)

In [ ]:
client.who_has()

In [ ]:
%%time
lr.fit(X_df)

In [ ]:
%%time
D, I = lr.kneighbors(X_df[0:10000], search_k)

In [ ]:
print(str(I.compute()))

In [ ]:
print(str(D.compute()))

In [ ]:
np.array(D.compute().as_gpu_matrix())